In [ ]:
import os
import numpy as np
from keras.utils import to_categorical

In [ ]:
root_folder = './commands/'

In [ ]:
command_labels = ['bongeszo', 'fazok', 'homerseklet_alap','homerseklet_vissza','klima_be','klima_le','melegem_van','mennyi_az_ido','vicc','villany_fel','villany_le' ]

In [ ]:
len(command_labels)

In [ ]:
features = []
labels = []

In [ ]:
import librosa
import numpy as np

In [ ]:
def preprocess_log_mel_spectrogram(file_path,n_mels=128,  duration=4.0):
    # Load the audio file
    
    waveform, sr = librosa.load(file_path, duration=duration)
    hop_length = int(sr * (0.02 - 0.01))
    # Extract the Log-Mel-spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(waveform, sr=sr, n_mels=n_mels, hop_length=hop_length)
    log_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

    return log_mel_spectrogram

In [ ]:
#input_shape=(128, 401, 1) input_shape=(n_mels, hop_length, 1)

In [ ]:
%%capture
for label_idx, label in enumerate(command_labels):
    command_folder = os.path.join(root_folder, label)
    files = os.listdir(command_folder)

    # Loop through the audio files in each command folder
    for file in files:
        file_path = os.path.join(command_folder, file)

        # Load and preprocess the Log-Mel-spectrogram (replace with your own preprocessing steps)
        mel_spectrogram = preprocess_log_mel_spectrogram(file_path)

        # Append the features and labels
        features.append(mel_spectrogram)
        labels.append(label_idx)

In [ ]:
print(len(features[0][0]))
print(len(features[1000][0]))
print(len(features[6000][0]))
print(len(features[9000][0]))
print(len(features[10000][0]))


In [ ]:
max_shape = max(feature.shape for feature in features)

# Create a new list to store the padded Log-Mel-spectrograms
padded_features = []
ff = 0
# Pad the Log-Mel-spectrograms to match the maximum shape
for feature in features:
    pad_width = [(0, max_shape[0] - feature.shape[0]), (0, max_shape[1] - feature.shape[1])]
    ff = pad_width
    padded_feature = np.pad(feature, pad_width, mode='constant')
    padded_features.append(padded_feature)

# Convert the padded features to a NumPy array
features = np.array(padded_features)
labels = np.array(labels)

train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Print the shapes of the training and validation sets
print("Training features shape:", train_features.shape)
print("Training labels shape:", train_labels.shape)
print("Validation features shape:", val_features.shape)
print("Validation labels shape:", val_labels.shape)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Convert the features and labels to NumPy arrays
features = np.array(features)
labels = np.array(labels)

# Split the data into training and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.2, random_state=42)

# Print the shapes of the training and validation sets
print("Training features shape:", train_features.shape)
print("Training labels shape:", train_labels.shape)
print("Validation features shape:", val_features.shape)
print("Validation labels shape:", val_labels.shape)

In [ ]:
model = 1

In [ ]:
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, TimeDistributed
 
model = Sequential()

    # CNN layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 401, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

    # LSTM layers
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, recurrent_dropout=0.1,return_sequences=False))

    # Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))

In [ ]:
model = Sequential()

    # CNN layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 401, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

    # Fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))

In [ ]:
# import tensorflow as tf
# tf.config.experimental_run_functions_eagerly(False)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]

In [ ]:

model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

In [ ]:
train_labels = to_categorical(train_labels, num_classes=11)

In [ ]:
batch_size = 64
epochs = 20

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.keras.losses.CategoricalCrossentropy()
metrics = [tf.keras.metrics.CategoricalAccuracy()]
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

model.fit(train_features, train_labels, batch_size=batch_size, epochs=epochs)

In [ ]:
model.save("convloss05catacc98.h5")


In [ ]:
tf.saved_model.save(model, "./saved_trained_models/")

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('./convloss05catacc98.h5')

In [ ]:
import librosa
import numpy as np

spec_to_predict = preprocess_log_mel_spectrogram('./test_voices/lekgondi_le2.wav')
spec_to_predict.shape

In [ ]:
# A modell elvárt input shape-je
desired_shape = model.input_shape

# Kiszámítjuk a szükséges padding-et az első két dimenzióra
pad_width = [(0, desired_shape[1] - spec_to_predict.shape[0]),
             (0, desired_shape[2] - spec_to_predict.shape[1])]

# Padding-et alkalmazunk a spektrogramra
padded_spec = np.pad(spec_to_predict, pad_width, mode='constant')

# Az eredmény shape-je már megfelel a modell elvárt input shape-jének
print(padded_spec.shape)



In [ ]:
spec_to_predict = np.expand_dims(padded_spec, axis=-1)
print(spec_to_predict.shape)


In [ ]:
spec_to_predict_batch = np.expand_dims(spec_to_predict, axis=0)
print(spec_to_predict_batch.shape)

In [ ]:
predictions = model.predict(spec_to_predict_batch)

print(predictions)

In [ ]:
predicted_class_index = np.argmax(predictions)

In [ ]:
predicted_class = command_labels[predicted_class_index]
print(predicted_class)

In [ ]:
import librosa
import numpy as np

waveformp, srp = librosa.load('./test_voices/hom_alap.wav', duration=4.0)
hop_length = int(srp * (4.0 / 588.0))
print(hop_length)
    # Extract the Log-Mel-spectrogram
mel_spectrogramPRE = librosa.feature.melspectrogram(waveformp, sr=srp, n_mels=128, hop_length=149)
log_mel_spectrogramPRE = librosa.amplitude_to_db(mel_spectrogramPRE, ref=np.max)

log_mel_spectrogramPRE = log_mel_spectrogramPRE.reshape(1, log_mel_spectrogramPRE.shape[0], log_mel_spectrogramPRE.shape[1], 1)

predictions = model.predict(log_mel_spectrogramPRE)

In [ ]:
log_mel_spectrogramPRE.shape[]

In [ ]:
log_mel_spectrogramPRE = log_mel_spectrogramPRE.reshape(1, log_mel_spectrogramPRE.shape[0], log_mel_spectrogramPRE.shape[1], 1)

In [ ]:
gg = np.pad(log_mel_spectrogramPRE, ff, mode='constant')

In [ ]:
predictions = model.predict(log_mel_spectrogramPRE)